# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_file = "Output/city.csv"

city_file_df = pd.read_csv(city_file)
city_file_df.head()


,Country,City,Latitude,Longitude,Temperature,Humidity,Clouds,Winds
0,ID,Labuhan,-6.8844,112.2051,76.57,89,94,8.72
1,NZ,Bluff,-46.6000,168.3333,54.81,83,100,10.49
2,AR,Ushuaia,-54.8000,-68.3000,53.26,43,20,17.27
3,ZA,Hermanus,-34.4187,19.2345,70.81,79,13,15.97
4,RU,Tymovskoye,50.8456,142.6550,-12.64,100,62,3.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = city_file_df[["Latitude", "Longitude"]].astype(float)
humidity = city_file_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
perf_weather_df = city_file_df.loc[city_file_df.Temperature >= 70][city_file_df.Humidity < 60][city_file_df.Clouds < 15][city_file_df.Winds < 5]
perf_weather_df.reset_index(drop = True, inplace = True)
perf_weather_df


C:\Users\Rome\AppData\Local\Temp/ipykernel_14884/4166830623.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_weather_df = city_file_df.loc[city_file_df.Temperature >= 70][city_file_df.Humidity < 60][city_file_df.Clouds < 15][city_file_df.Winds < 5]


,Country,City,Latitude,Longitude,Temperature,Humidity,Clouds,Winds
0,US,Sun City West,33.6620,-112.3413,79.56,15,0,3.44
1,AR,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,95.79,25,6,1.88
2,US,Saint George,37.1041,-113.5841,73.87,19,0,0.00
3,SS,Raja,8.4596,25.6780,75.29,13,8,3.96
4,AR,San Luis,-33.2950,-66.3356,88.72,21,1,4.76
5,CM,Rey Bouba,8.6724,14.1786,81.73,12,3,4.27
6,SA,Makkah al Mukarramah,21.4267,39.8261,73.83,34,0,4.21
7,BR,Santa Maria,-29.6842,-53.8069,83.50,30,0,2.30
8,BR,Ituporanga,-27.4144,-49.6025,74.53,45,8,1.70
9,CM,Tcholliré,8.4022,14.1698,76.44,14,3,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = perf_weather_df
hotel_df["Hotel Name"] = ""
hotel_df

,Country,City,Latitude,Longitude,Temperature,Humidity,Clouds,Winds,Hotel Name
0,US,Sun City West,33.6620,-112.3413,79.56,15,0,3.44,
1,AR,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,95.79,25,6,1.88,
2,US,Saint George,37.1041,-113.5841,73.87,19,0,0.00,
3,SS,Raja,8.4596,25.6780,75.29,13,8,3.96,
4,AR,San Luis,-33.2950,-66.3356,88.72,21,1,4.76,
5,CM,Rey Bouba,8.6724,14.1786,81.73,12,3,4.27,
6,SA,Makkah al Mukarramah,21.4267,39.8261,73.83,34,0,4.21,
7,BR,Santa Maria,-29.6842,-53.8069,83.50,30,0,2.30,
8,BR,Ituporanga,-27.4144,-49.6025,74.53,45,8,1.70,
9,CM,Tcholliré,8.4022,14.1698,76.44,14,3,1.01,


In [70]:
for index, row in hotel_df.iterrows(): 
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        lat = row["Latitude"]
        lng = row["Longitude"]

        params = {
            "keyword":"hotel",
            "radius": 5000,
            "key": g_key
        }

        params["location"] = f"{lat}, {lng}"


        hotel_data = requests.get(base_url, params=params).json()

        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,Country,City,Latitude,Longitude,Temperature,Humidity,Clouds,Winds,Hotel Name
0,US,Sun City West,33.6620,-112.3413,79.56,15,0,3.44,Days Inn & Suites by Wyndham Surprise
1,AR,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,95.79,25,6,1.88,Hotel Gualok
2,US,Saint George,37.1041,-113.5841,73.87,19,0,0.00,Red Lion Hotel & Conference Center St. George
3,SS,Raja,8.4596,25.6780,75.29,13,8,3.96,NaN
4,AR,San Luis,-33.2950,-66.3356,88.72,21,1,4.76,GRAN HOTEL SAN LUIS GOLD
5,CM,Rey Bouba,8.6724,14.1786,81.73,12,3,4.27,NaN
6,SA,Makkah al Mukarramah,21.4267,39.8261,73.83,34,0,4.21,"Makkah Clock Royal Tower, A Fairmont Hotel"
7,BR,Santa Maria,-29.6842,-53.8069,83.50,30,0,2.30,Hotel Appel
8,BR,Ituporanga,-27.4144,-49.6025,74.53,45,8,1.70,Hotel Jardim Bulevard
9,CM,Tcholliré,8.4022,14.1698,76.44,14,3,1.01,NaN


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [76]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))